In [15]:
import keras
from keras import layers
from keras.utils import np_utils
from keras.datasets import mnist
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from keras import optimizers

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

In [10]:
mean = np.mean(x_train, axis = (0,1,2,3))
std = np.std(x_train, axis = (0,1,2,3))
x_train = (x_train - mean)/ (std + 1e-7)
x_test = (x_test - mean)/ (std + 1e-7)

In [11]:
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [12]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0

In [36]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
rotation_range = 15,  
width_shift_range = .1,  
height_shift_range = .1,
horizontal_flip = True) 

datagen.fit(x_train)

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam", 
              metrics=['accuracy'])

In [40]:
batch_size = 64
opt_rms = tf.keras.optimizers.RMSprop(lr = .0001, decay = 1e-6)
model.compile(loss = "categorical_crossentropy", optimizer = opt_rms,
             metrics = ["accuracy"])
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                   steps_per_epoch = x_train.shape[0]//batch_size, epochs = 12,
                   verbose = 1, validation_data=(x_test, y_test))

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_25524/4024835136.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),


Epoch 1/12
937/937 [==============================] - 258s 273ms/step - loss: 0.3572 - accuracy: 0.8871 - val_loss: 0.0699 - val_accuracy: 0.9765
Epoch 2/12
937/937 [==============================] - 246s 262ms/step - loss: 0.1897 - accuracy: 0.9422 - val_loss: 0.0563 - val_accuracy: 0.9818
Epoch 3/12
937/937 [==============================] - 245s 261ms/step - loss: 0.1475 - accuracy: 0.9539 - val_loss: 0.0589 - val_accuracy: 0.9808
Epoch 4/12
937/937 [==============================] - 242s 258ms/step - loss: 0.1276 - accuracy: 0.9608 - val_loss: 0.0518 - val_accuracy: 0.9837
Epoch 5/12
937/937 [==============================] - 242s 258ms/step - loss: 0.1135 - accuracy: 0.9654 - val_loss: 0.0500 - val_accuracy: 0.9837
Epoch 6/12
937/937 [==============================] - 240s 256ms/step - loss: 0.1032 - accuracy: 0.9684 - val_loss: 0.0405 - val_accuracy: 0.9866
Epoch 7/12
937/937 [==============================] - 234s 250ms/step - loss: 0.0954 - accuracy: 0.9708 - val_loss: 0.0496 -

In [41]:
scores = model.evaluate(x_test, y_test, batch_size = 128, verbose = 1)
print('\nTest Results: %.3f loss: %.3f' %(scores[1]*100,scores[0]))

79/79 [==============================] - 12s 149ms/step - loss: 0.0424 - accuracy: 0.9862

Test Results: 98.620 loss: 0.042
